In [18]:
from konlpy.tag import Twitter
from sklearn.preprocessing import normalize
from scipy.sparse import dok_matrix
from collections import Counter
from random import shuffle
import numpy as np
import json
import pickle

twitter = Twitter()

In [21]:
with open("data/2017_03.json") as f:
    data = json.load(f)

shuffle(data)
data = data[:10000]

In [23]:
data[0]

{'date': '2017-03-06',
 'text': '특검 최종 수사결과 발표\n\n야당 "검찰, 남은 과제 철저히 수사해야"\n\n친박 자유한국당 "여론 편승 수사결과" 비난\n\n‘박근혜 게이트’를 수사한 박영수 특별검사팀이 6일 박근혜 대통령이 전면 부인해온, ‘비선실세’ 최순실씨와의 공모를 통해 뇌물을 수수한 혐의 등을 사실로 판단한 최종 수사결과를 발표했다.\n\n이에 대해 야당들은 일제히 박 대통령을 비판하며 향후 특검의 수사를 이어갈 검찰에 박 대통령의 대면조사, 세월호 7시간 등에 대한 공정한 수사를 당부했다. 반면 ‘친박당’ 자유한국당은 “편향된 여론에 편승한 언론플레이”라고 비난했다.\n\n고용진 더불어민주당 대변인은 이날 국회 브리핑에서 “최순실과 비선실세들이 사익을 위해 국정을 농단했고 최순실과 박 대통령은 공범임이 명명백백하게 드러났다”며 “이는 끝이 아닌 진실 규명의 새로운 시작이 되어야 한다”고 밝혔다.\n\n고 대변인은 “황교안 권한대행의 거부로 특검팀의 수사기간이 만료돼 삼성을 제외한 재벌대기업의 정경유착, 우병우 전 민정수석과 문고리 권력에 대한 수사는 결국 미완의 과제로 남겨졌다”며 “검찰이 특검팀의 수사를 이어받아 남은 과제들을 제대로 수사하지 못한다면 검찰에 대한 국민적 회의와 불신은 걷잡을 수 없이 확산될 것”이라고 경고했다.\n\n또한 “자신의 약속을 뒤집고 대면 조사를 거부하는 등 조직적으로 특검수사를 방해한 박근혜 대통령과 청와대의 행태는 결코 용서받지 못할 것”이라며 “검찰은 특검이 하지 못했던 청와대에 대한 압수수색과 박 대통령에 대한 대면조사를 즉각 추진해야 한다”고 촉구했다.\n\n장진영 국민의당 대변인도 논평을 내고 특검의 성과를 긍정적으로 평가하면서도 “황교안 총리의 특검연장 불승인, 정세균 의장의 특검법 개정안 직권상정 거부로 대통령 대면조사, 우병우 구속, 롯데, CJ 수사 등 특검수사의 화룡점정을 찍지 못한 것이 무엇보다 아쉬울 뿐”이라고 밝혔다.\n\n장 대변인은 “마무리는 검찰, 법원, 헌법

In [3]:
titles = set()
articleList = []
articleNum = 0

for article in data:
    if article['title'] not in titles:
        # 기사 중복되지 않는 것들만 처리
        titles.add(article['title'])
        articleList.append(article['text'])
        articleNum += 1

del data

In [4]:
tags = set(['Noun', 'Verb', 'Adjective'])
spamWords = set([
    '하다', '되다', '이다', '전재', '배포', '무단', '기사', '금지', '사진', '보기', '뉴스', '있다',
    '돼다', '앵커', '위해', '라며', '그렇다', '보다', '되어다', '기자', '하는', '이런', '그런', '하는', '이상'
])

word_count = Counter()

for i, article in enumerate(articleList):
    if i % 1000 == 0:
        print(i)

    words = []
    pos = twitter.pos(article, norm=True, stem=True)
    word_count.update([word for word, tag in pos if tag in tags and len(word) >1 and word not in spamWords])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [17]:
index2voca = set()
index2voca.update(['안녕', '하하', '안녕'])
print(index2voca)

{'하하', '안녕'}


In [6]:
wordsList = []
raw_text = []
index2voca = set()
for i, article in enumerate(articleList):
    if i % 1000 == 0:
        print(i)

    words = []
    pos = twitter.pos(article, norm=True, stem=True)
    words = [word for word, tag in pos if word_count[word] >= 10]
    if len(words) >= 10:
        index2voca.update(words)
        wordsList.append(words)
        raw_text.append(article)

index2voca = list(index2voca)
voca2index = {w: i for i, w in enumerate(index2voca)}

print(len(index2voca))
print(len(wordsList))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
11267
8805


In [9]:
voca2index['서울']

1011

In [13]:
# term-document matrix를 sparse matrix로 생성
tdm = dok_matrix((len(wordsList), len(index2voca)), dtype=np.float32)
print(tdm.shape)

for i, words in enumerate(wordsList):
    for word in words:
        tdm[i, voca2index[word]] += 1

tdm = tdm.tocsr()
# 각 document별로 l2-normalize
tdm = normalize(tdm)

(8805, 11267)


In [15]:
with open("data/preprocessed_data.pkl", 'wb') as f:
    pickle.dump({'index2voca': index2voca, 'voca2index': voca2index, 'wordsList': wordsList, 'tdm': tdm, 'raw_text': raw_text}, f)